## ** Building a large scale job Recommendation Engine using Implicit data in pyspark **

In [1]:
import os
import math
import datetime
import pyspark.sql.functions as sf
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
from pyspark import SparkConf, SparkContext
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
from pyspark import SparkContext, SparkConf, SQLContext
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.ml.recommendation import ALS
# from pyspark.sql import Row
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.sql.types import TimestampType

In [2]:
# conf = SparkConf().setAppName("jobRecommendationEngine")
# sc = SparkContext(conf=conf)
sc

<SparkContext master=local[*] appName=PySparkShell>

In [27]:
# load job Clicks file into rdd
datasets_path=os.getcwd() + "/RE_data"
ratings_file = os.path.join(datasets_path, 'job_clicks.csv')
ratings_raw_data = sc.textFile("file:///" + ratings_file)
ratings_raw_data_header = ratings_raw_data.take(1)[0]
ratings_data = ratings_raw_data.filter(lambda line: line != ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),int(float(tokens[2])))).cache()

print ("train size is ", ratings_data.count())
print('\nColumns are:', ratings_raw_data_header)
ratings_data.take(3)

train size is  100004

Columns are: userId,jobId,Clicks


[(1, 31, 3), (1, 1029, 66), (1, 1061, 93)]

In [28]:
# load jobs category file into rdd
jobs_file = os.path.join(datasets_path, 'jobs.csv')
jobs_raw_data = sc.textFile("file:///" + jobs_file)
jobs_raw_data_header = jobs_raw_data.take(1)[0]

jobs_data = jobs_raw_data.filter(lambda line: line!=jobs_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1])).cache()
print('\nColumns are:', jobs_raw_data_header)
jobs_data.take(3)


Columns are: jobID,job_category


[(1, 'IT'), (2, 'IT'), (3, 'IT')]

### Split data into train, validation and test datasets

In [4]:
rddTraining, rddValidating, rddTesting = ratings_data.randomSplit([6,2,2], seed=1001)

#Add user ratings in the training model
nbValidating = rddValidating.count()
nbTesting    = rddTesting.count()

print("Training: %d, validation: %d, test: %d" % (rddTraining.count(), nbValidating, rddTesting.count()))

Training: 59902, validation: 20112, test: 19990


### Model Training

Here, I am using RMSE but for these kind of problems where we have implicit
features, it is better to use ** Mean Percentage Ranking (MPR) **

In [5]:
#[START how_far]
def howFarAreWe(model, against, sizeAgainst):
    againstNoRatings = against.map(lambda x: (int(x[0]), int(x[1])) )
    againstWiRatings = against.map(lambda x: ((int(x[0]),int(x[1])), int(x[2])) )
    predictions = model.predictAll(againstNoRatings).map(lambda p: ( (p[0],p[1]), p[2]) )
    predictionsAndRatings = predictions.join(againstWiRatings).values()    
    return sqrt(predictionsAndRatings.map(lambda s: (s[0] - s[1]) ** 2).reduce(add) / float(sizeAgainst))
#[END how_far]

In [7]:
#finding best set of parameters
ranks  = [5,10,15,20]
reguls = [0.1, 1,10]
iters  = [5,10,20]
alpha = [10, 20, 40]

finalModel = None
finalRank  = 0
finalRegul = float(0)
finalIter  = -1
finalDist   = float(300)
finalAlpha = float(0)

#[START train_model]
for cRank, cRegul, cIter, cAlpha in itertools.product(ranks, reguls, iters, alpha):
    model = ALS.trainImplicit(rddTraining, cRank, cIter, float(cRegul),alpha=float(cAlpha))
    dist = howFarAreWe(model, rddValidating, nbValidating)
    if dist < finalDist:
        print(cIter, cRank,cAlpha,cRegul)
        print("Best so far:%f" % dist)
        finalModel = model
        finalRank  = cRank
        finalRegul = cRegul
        finalIter  = cIter
        finalDist  = dist
        finalAlpha  = cAlpha 
#[END train_model]

print("Rank %i" % finalRank) 
print("Regul %f" % finalRegul) 
print("Iter %i" % finalIter)  
print("Dist %f" % finalDist) 
print("Alpha %f" % finalAlpha) 


5 5 10 0.1
Best so far:55.682865
5 5 20 0.1
Best so far:55.664132
5 5 40 0.1
Best so far:55.656700
10 5 20 0.1
Best so far:55.646255
10 5 40 0.1
Best so far:55.643666
20 5 20 0.1
Best so far:55.642589
10 5 20 1
Best so far:55.638122
10 5 40 1
Best so far:55.638103
20 5 20 1
Best so far:55.637189
20 5 40 1
Best so far:55.627915
10 5 40 10
Best so far:55.622670
Rank 5
Regul 10.000000
Iter 10
Dist 55.622670
Alpha 40.000000


In [9]:
model = ALS.trainImplicit(rddTraining, rank=finalRank, iterations=finalIter, lambda_= float(finalRegul),alpha=float(finalAlpha))
# Calculate all predictions
rddTesting_withoutclicks = rddTesting.map(lambda r: ((r[0], r[1])))
predictions = model.predictAll(rddTesting_withoutclicks).map(lambda r: ((r[0], r[1]), (r[2])))
predictions.take(3)
# user id, node_id, actual clickss,pred clickss -> df below
rates_and_preds = rddTesting.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions) 
rates_and_preds.take(3)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 57.19030098939986


In [10]:
from pyspark.sql import SparkSession
x = rates_and_preds.map(lambda x : (x[0][0],x[0][1],x[1][0],x[1][1]))
hasattr(x, "toDF")
x.toDF().show(4)

+---+----+----+------------------+
| _1|  _2|  _3|                _4|
+---+----+----+------------------+
|309|3671|32.0|1.0062152515438365|
| 42|2916|68.0|0.9550179359043851|
|547|3741|27.0|0.9653563646571577|
|243|2617|11.0|1.0072970809312765|
+---+----+----+------------------+
only showing top 4 rows



### Get total clicks and average clicks given to each job by different users

In [11]:
def get_counts_and_averages(ID_and_ratings_tuple):    
    nratings = len(ID_and_ratings_tuple[1]) 
    return ID_and_ratings_tuple[0], (nratings, sum([float(val) for val in ID_and_ratings_tuple[1]])/nratings)

In [14]:
job_ID_with_ratings_RDD = (ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
job_ID_with_ratings_RDD_updated = job_ID_with_ratings_RDD.map(lambda x : (x[0], list(x[1])))
job_ID_with_avg_ratings_RDD = job_ID_with_ratings_RDD_updated.map(get_counts_and_averages)  # count and average rating
job_rating_counts_RDD = job_ID_with_avg_ratings_RDD.map(lambda x: (int(x[0]), x[1][0]))    # rating count per job
job_rating_counts_RDD.cache()
job_rating_counts_RDD.take(3)

[(2, 107), (97328, 1), (4, 13)]

In [18]:
# get user-wise jobs clicked
all_users_ratings_RDD = ratings_data.map(lambda x: (x[0], x[1])).groupByKey()
all_users_ratings_RDD = all_users_ratings_RDD.map(lambda x : (x[0], list(x[1])))    # jobs clicked by each user

### finding unrated jobs by each user- we will use this set for model's prediction/recommendations
job_ids = set(jobs_data.map(lambda x : x[0]).toLocalIterator()) # list of all job ids
unrated_jobs_RDD = all_users_ratings_RDD.map(lambda x: (x[0], list((job_ids) - set(x[1]))))

# #create user_id and unrated job id pairs
unrated_userjobs_RDD = unrated_jobs_RDD.flatMap(lambda x : [(x[0],i) for i in x[1]])

# # #model predictions for each user and not clicked job pairs
recommendations_RDD = model.predictAll(unrated_userjobs_RDD)
recommended_jobs_rating_RDD = recommendations_RDD.map(lambda x: (x.product,(x.user, x.rating)))
recommended_jobs_rating_RDD.cache()
print (recommended_jobs_rating_RDD.take(10))

[(161944, (312, 0.2807878695601902)), (161944, (296, 0.10213990447938404)), (161944, (96, -0.1034895934494533)), (161944, (440, -0.15559400922318423)), (161944, (536, 0.0585588119071474)), (161944, (600, 0.05502822506654903)), (161944, (16, -0.19563889120453504)), (161944, (320, -0.0563454939003778)), (161944, (216, -0.030783395765506455)), (161944, (408, 0.20569715889025542))]


### Joining job title and total number of clicks received by each job for further filtering recommendations

In [19]:
# #     # converting id into int for job_clicks_count RDD to perform join
# job_clicks_counts_RDD_updated = job_clicks_counts_RDD.map(lambda x: (int(x[0]), x[1]))

# join job name with job id, predicted rating for job and total number of ratings received by each job
recommendations_rating_title_and_count_RDD = recommended_jobs_rating_RDD.join(jobs_data).join(job_rating_counts_RDD)
recommendations_rating_title_and_count_RDD = recommendations_rating_title_and_count_RDD.map(lambda r: (r[0], r[1][0][1], r[1][0][0][0],round(r[1][0][0][1],2),r[1][1]))
recommendations_rating_title_and_count_RDD = recommendations_rating_title_and_count_RDD.map(lambda x: (x[2],(x[0],x[1], x[3],x[4])))
recommendations_rating_title_and_count_RDD.take(3)

[(312, (27648, 'Business Intelligence', 0.23, 1)),
 (296, (27648, 'Business Intelligence', -0.19, 1)),
 (96, (27648, 'Business Intelligence', 0.37, 1))]

## ** Top 5 recommendations **

In [22]:
# filter only those jobs which have been clicked by atleast 20 users
# take only top5 jobs by sorting based on preference confidence
top_jobs = recommendations_rating_title_and_count_RDD.groupBy(lambda x : x[0])\
                               .map(lambda x : list(x[1]))\
                               .map(lambda r: [i for i in r if i[1][3] > 20])\
                               .map(lambda a: [i for i in sorted(a, key=lambda x: -x[1][2])[:5]])   

#preparing dataframe to insert in Database
rec_jobs_df = top_jobs.map(lambda x: [(i[0],i[1][0],i[1][1],i[1][2]) for i in x]).flatMap(lambda x: x).toDF()\
                                .withColumnRenamed("_1", "user_id")\
                                .withColumnRenamed("_2", 'job_recommendations')\
                                .withColumnRenamed("_3", 'job_category')\
                                .withColumnRenamed("_4", 'preference_confidence')\
                                .withColumnRenamed("_5", "total_clicks")
                
# #final recommendation engine dataframe to be saved in Database
final_df_rec_eng = rec_jobs_df.withColumn("rec_date", sf.lit(datetime.datetime.now()).cast(TimestampType()))   
final_df_rec_eng = final_df_rec_eng.withColumn("rec_number", sf.row_number().over(Window.partitionBy("user_id").orderBy(desc("preference_confidence"))))    
final_df_rec_eng.show(15)

+-------+-------------------+--------------------+---------------------+--------------------+----------+
|user_id|job_recommendations|        job_category|preference_confidence|            rec_date|rec_number|
+-------+-------------------+--------------------+---------------------+--------------------+----------+
|     26|              41569|                  IT|                 1.13|2019-02-17 00:58:...|         1|
|     26|             116797|           Marketing|                 1.07|2019-02-17 00:58:...|         2|
|     26|             106489|              Retial|                 1.07|2019-02-17 00:58:...|         3|
|     26|              76093|Business Intellig...|                 1.06|2019-02-17 00:58:...|         4|
|     26|               7022|                  HR|                 1.06|2019-02-17 00:58:...|         5|
|     29|                588|                  HR|                 1.07|2019-02-17 00:58:...|         1|
|     29|                349|                  IT|     

**Before showing the top 5 recommendations, we can filter them based on job category to show only 
recommendations from the categories users have previously looked at.
Also, Here, I am using RMSE but for these kind of problems where we have implicit
features, it is better to use Mean Percentage Ranking (MPR) **

### Saving the model

In [ ]:
# from pyspark.mllib.recommendation import MatrixFactorizationModel

# model_path = os.path.join('..', 'models', 'job_lens_als')

# # Save and load model
# model.save(sc, model_path)
# same_model = MatrixFactorizationModel.load(sc, model_path)